## Dependences

In [69]:
!pip install tqdm

## Timer

In [70]:
def print_red(text):
    print('\x1b[31m' + text + '\x1b[0m')
    
def print_yellow(text):
    print('\x1b[33m' + text + '\x1b[0m')

def print_green(text):
    print('\x1b[32m' + text + '\x1b[0m')

def print_pink(text):
    print('\x1b[35m' + text + '\x1b[0m')

def print_cyan(text):
    print('\x1b[36m' + text + '\x1b[0m')

## Ranking all annotations
Run all the possible combinations of annotations with different models and stores the winners in the variable *rounds*. There can be multiple winners.

***An annotation is a winner if it has the best value above all of the other and the best time.***

##### Example of *rounds*

|  | Ann_1 | Ann_2 | Ann_3 | Ann_4 | ... | Ann_n |
| --- | --- | --- | --- | --- | --- | --- |
| Ins_1 | 0 | 0 | 1 | 1 | ... | 0 |
| Ins_2 | 0 | 1 | 0 | 0 | ... | 1 |
| Ins_3 | 1 | 0 | 1 | 0 | ... | 0 |
| ... | ... | ... | ... | ... | ... | ... |
| Ins_m | 1 | 0 | 1 | 0 | ... | 0 |

In [71]:
import numpy as np
import os   
import subprocess 
import random
import math

# Visualfrom tqdm
from tqdm import tqdm

def select_winners(mnts, times):
    winners = [1] * len(mnts)
    min_mnt = min(mnts)
    min_time = min(times)
    # print_green(f'min_mnt {min_mnt} min_time {min_time}.')
    for i, mnt in enumerate(mnts):
        if math.isinf(mnt):
            winners[i] = 0
        else:
            if mnt == min_mnt:
                if times[i] != min_time:
                    winners[i] = 0
            else:
                winners[i] = 0

    return winners

def annotations_winners(index, models_folder, instances_folder, timeout_mzn, timeout):
    annotations = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]
    instance = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')][index]
    
    mnts = [0] * len(annotations)
    times = [0] * len(annotations)
    timeout_mzn = timeout_mzn*1000
    
    for i, annotation in tqdm(enumerate(annotations), desc='Processing Instance '+str(index), unit='items'):
        
        # Run the minizinc command for the current annotation and data file  
        cmd = f"minizinc --solver HiGHS --output-time {annotation} {instance} --output-time --solver-time-limit {timeout_mzn}"
        proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        try: 
            # Get output from fzn
            stdout, stderr = proc.communicate(timeout=timeout) 
            stdout = stdout.decode()
            try:
                # Store value mnt (temp).
                start = stdout.find('mnt = ')+6
                end = stdout.find(';',start)
                mnt = int(stdout[start:end])
                # Store value time (temp).
                time = float(stdout.split(' ')[-2])
            except:
                mnt = float('inf')
                time = float('inf')
            # print_green(f'{annotation[10:-4]} with {instance[3:-4]} SOLVED: {mnt} in {time:.2f} seconds.')

        except subprocess.TimeoutExpired: 
            print_red(f'Error from minizinc: Stopping.')                
            mnt = float('inf')
            time = float('inf')
            if os.name == 'nt': # If the os is Windows
                subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
            else:
                os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group
                
        mnts[i] = mnt
        times[i] = time
    # print('mnts',mnts)
    # print('times',times)
    return select_winners(mnts,times)

n_instances = 500
rounds = np.zeros((n_instances, len([os.path.join('../models', f) for f in os.listdir('../models') if f.endswith('.mzn')])))

# loop through the list with a progress bar
for i in range(0, n_instances):
    instance = random.randint(0,n_instances-1)
    rounds[i]=np.array(annotations_winners(instance, '../models', '../instances',5, 10))
    print_yellow(f'Finished: {i}')
print_green(f'Done!')

Processing Instance 444: 54items [00:15,  3.52items/s]


Finished: 0


Processing Instance 168: 54items [00:10,  5.11items/s]


Finished: 1


Processing Instance 352: 54items [00:18,  2.95items/s]


...

Finished: 207


Processing Instance 38: 54items [00:10,  4.97items/s]


Finished: 208


Processing Instance 61: 13items [00:03,  3.37items/s]


KeyboardInterrupt: 

## Store and transform *rounds*

Save *rounds* to another file, sum all the rounds to get the best annotation.

In [ ]:
path = '../models/rounds.txt'
with open(path, 'w', encoding='utf8') as file_object:
    file_object.write(str(rounds.tolist()))

rounds = ''
with open('../models/rounds.txt', encoding="utf8") as file_object:
        rounds = np.array(eval(file_object.read()))
        
print_pink("rounds")
print(rounds)

# sum the rounds
scores = np.sum(rounds, axis=0)
scores = scores.astype(int)

# print the scores
print_pink("\nFinal score:")
print(scores)

## Plot the scores on a bar chart.

In [ ]:
import matplotlib.pyplot as plt 

# Define figure size
fig, ax = plt.subplots(figsize=(8, 12))

models_folder = '../models'  
models = [os.path.join(models_folder, f) for f in os.listdir(models_folder) if f.endswith('.mzn')]
# Filter models and times
models = [model.replace("../models\\", "").replace(".mzn", "") for model in models]

# Set different color for bars with highest values
max_idx = np.max(scores)
colors = ['#b03060' if score != max_idx else 'green' for score in scores]
# Use barh function to create horizontal bar chart
ax.barh(models, scores, color=colors)

# Set x-axis label and y-axis label
ax.set_xlabel('Scores (Won rounds)')
ax.set_ylabel('Models')

# Add value labels next to each bar
for i, v in enumerate(scores):
    ax.text(v, i, str(v), color='black', ha='left', va='center')

# Show the plot
plt.show()
